In [ ]:
# !pip install pyyaml==5.1
# !pip install "git+https://github.com/facebookresearch/detectron2.git"

In [ ]:
import torch, detectron2
from pathlib import Path
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

Import Detectron

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from livecell_tracker.segment.detectron_utils import gen_cfg

In [ ]:
from detectron2.data.datasets import register_coco_instances

train_data_path = r"""."""
train_json_path = r"""./notebook_results/labelme2coco_test_output_8bit_png__ver0.0.2/train.json"""

val_data_path = r"""."""
val_json_path = r"""./notebook_results/labelme2coco_test_output_8bit_png__ver0.0.2/val.json"""
register_coco_instances("deepfashion_train", {}, train_json_path, train_data_path)
register_coco_instances("deepfashion_val", {}, val_json_path, val_data_path)

In [ ]:
output_dir = os.path.join("./segment_train_results/", "detectron_train_output__ver0.0.2")
os.makedirs(output_dir, exist_ok=True)

## Detectron config

In [ ]:
cfg = gen_cfg(output_dir=output_dir)

In [ ]:
import torch
print(torch.version)
print(torch.cuda.is_available())

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
os.makedirs(output_dir, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

Run training

In [ ]:
import time
start = time.time()
trainer.train()
completion_interval = time.time() - start
print("Training time: ", completion_interval)

## Evaluation

In [ ]:
cfg = gen_cfg(output_dir=output_dir)
cfg.OUTPUT_DIR = output_dir
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("deepfashion_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
import livecell_tracker.segment.datasets

dir_path = Path('..//cxa-data//june_2022_data_8bit_png/day0_Notreat_Group 1_wellA1_RI_MIP_stitched')
# imgs = livecell_tracker.segment.datasets.LiveCellImageDataset(dir_path, ext="tif")
imgs = livecell_tracker.segment.datasets.LiveCellImageDataset(dir_path, ext="png")

In [ ]:
raw_img1 = imgs[50][:, :, np.newaxis]
# raw_img1 = normalize_img_by_zscore(raw_img1)

In [ ]:
def detectron_predict_single_img(img):
    outputs = predictor(img)
    return outputs

def detectron_predict_and_visualize_img(img):
    outputs = detectron_predict_single_img(img)
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    figure = plt.figure(figsize=(20, 20), dpi=80)
    plt.imshow(v.get_image()[:, :, ::-1])

In [ ]:
# detectron_predict_single_img(raw_img1)
detectron_predict_and_visualize_img(raw_img1)